[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aerosense-ai/notebooks/blob/main/pre_process.ipynb)

In [ ]:
!python --version

Python 3.7.15


In [ ]:
#install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9 python3.9-dev python3.9-distutils libpython3.9-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

# install pip
!sudo apt-get install python3-pip
!python -m pip install --upgrade pip

#check python version
!python --version

# Dependences and new code

In [ ]:
!pip install git+https://github.com/aerosense-ai/aerosense-tools.git@feature/add-pre-process

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import datetime as dt

from aerosense_tools.queries import BigQuery
from aerosense_tools import plots
from aerosense_tools.preprocess import RawSignal

In [23]:
import pandas as pd

In [24]:
pd.__version__

'1.3.5'

In [27]:
sensor_types_metadata = {
    "accelerometer": {
        "description": "Accelerometer Measurement Series",
        "variable": "Acceleration [m/s²]",
        "sensors": ["Acc. Direction 1", "Acc. Direction 2", "Acc. Direction 3"],
        },
    "barometer": {
        "description": "Absolute Barometer Measurement Series",
        "variable": "Pressure [Pa]",
        "sensors": ["Abs. Baro. Sensor {}".format(i) for i in range(40)],
    },
    "barometer_thermometer": {
        "description": "Temperature from Absolute Barometer Sensor Measurement Series",
        "variable": "Temperature [C]",
        "sensors": ["Temp. Sensor {}".format(i) for i in range(40)]
    },
    "connection_statistics": {
        "description": "Connection Statistics",
        "variable": [
            "Transmitting Power [W]",
            "Allocated Heap Memory [Bytes]",
            "Received signal strength indication: Raw [dBm]",
            "Received signal strength indication: Filtered [dBm]"
        ],
        "sensors": ["Conn. Stat. {}".format(i) for i in range(4)]
    },
    "differential_barometer": {
        "description": "Differential Barometer Measurement Series",
        "variable": "Differential Pressure [Pa]",
        "sensors": ["Diff. Baro. Sensor {}".format(i) for i in range(5)],
        },
    "gyroscope": {
        "description": "Gyroscope Measurement Series",
        "variable": "Angular Velocity [s⁻¹]",
        "sensors": ["Gyro. Direction 1", "Gyro. Direction 2", "Gyro. Direction 3"],
        },
    "magnetometer": {
        "description": "Magnetometer Measurement Series",
        "variable": "Magnetic Momemnt [A⋅m²]",
        "sensors": ["Mag. Direction 1", "Mag. Direction 2", "Mag. Direction 3"],
        },
}

In [5]:
client = BigQuery()

# Data overview
Before analysing all the data individually, we need to know what we have a bit more directly and visually

## labbook

we want to know:
- when the data has been recorded: *start_time end_time*
- how good the data is: what type of sensors do we have recorded + points to discard because faulty data (use Yuriy's cleaning process) and make a ratio of discarded points vs complete length, rssi as well?
- what were the wind conditions: wind speed (how?), rotational speed (make a simple algo on IMU to get an idea of rotational speed), average_temperature (average value from the IMU or pressure sensors?)

index | start_time | end_time | baros | IMU | mics | diffP | ratio good data | average blade rotational speed | temperature | zero_val | rssi_mean?
-- | --| -- | -- | --| --|--|-- | -- | -- | -- |--
int? | date | date | boolean | boolean | boolean | boolean | double | double | double | boolean | double

From that we will have an overview to be able to look at relevant data more efficiently.
Questions:
- How to present this?
- Where to store it?
- It should be a table that can be concatenated as soon new measurements are coming, with possibility to add some filtering

## Other parameters to be stored somewhere

- For each sensor node: Blade specs (at one specific location or more general?)
	- shape
	- chord length
- Data about baros
	- baro_index
	- index from LE (+ on suction side, - on pressure side)
	- position along the chord
	- radial position (along the blade)
	- Coeff for pressure and temperature that could be set individually for each of them
- Data about mics
	- baro_index (00 the one in the corner, i0 for those installed streamwise and 0j for those installed spanwise)
	- distance between each sensor from the previous ones (streamwise and  spanwise)
	- their positions along the chord and along r(?)
	- Coeffs for microphones (unique value for all of them might be sufficient)
- Data about IMU
	- position of IMU on the chord
	- orientation of IMU (Matrix of rotation of the data or angle to use complex/quaternion values)

## Some general functions to be added (but not urgent)
1. From temperature and atmospheric pressure estimate:
  - density
  - kinematic and dynamic viscosity

2. Give normal of one point from the blade section:
  - for barometers for example, it's helpful to integrate pressure.
  - need wing shape as reference


In [ ]:
#Get the time serties for barometer by running a SQL query
query_string = f"""
      SELECT datetime
      FROM `greta.sensor_data`
      WHERE sensor_type_reference="barometer" and
      node_id="2" and
      datetime between "2022-07-01 00:00:00.0" and "2022-09-01 00:00:00.0"
      ORDER BY datetime ASC
      """

baros_sample_times=client.query(query_string)

In [ ]:
# Not sure if this can be done more elegantly. 
# And, surely, can be done directly with SQL (I just don't dare).

threshold = dt.timedelta(seconds=60)
baros_gaps = baros_sample_times['datetime'].diff() > threshold

session_starts = baros_sample_times[baros_gaps]
session_ends=baros_sample_times.iloc[session_starts.index-1]

first_session_start = pd.DataFrame({"datetime":[baros_sample_times['datetime'][0]]})
last_session_end = pd.DataFrame({"datetime":[baros_sample_times['datetime'].iat[-1]]})

session_starts=pd.concat([first_session_start, session_starts])
session_ends=pd.concat([session_ends, last_session_end])

sessions = pd.DataFrame({"start_time":session_starts['datetime'].reset_index(drop=True), "end_time":session_ends['datetime'].reset_index(drop=True)})
sessions['session_duration']=sessions["end_time"]-sessions["start_time"]

In [ ]:
sessions

,start_time,end_time,session_duration
0,2022-07-14 11:06:11.715167,2022-07-14 11:06:21.944842,0 days 00:00:10.229675
1,2022-07-21 09:18:33.627776,2022-07-21 09:19:11.496264,0 days 00:00:37.868488
2,2022-07-21 10:39:21.971243,2022-07-21 10:41:55.604312,0 days 00:02:33.633069
3,2022-07-21 10:43:46.818994,2022-07-21 10:44:22.029420,0 days 00:00:35.210426
4,2022-07-21 11:00:03.773041,2022-07-21 11:00:37.015361,0 days 00:00:33.242320
5,2022-07-21 11:04:50.027605,2022-07-21 11:05:03.783538,0 days 00:00:13.755933
6,2022-07-21 13:00:03.960630,2022-07-21 13:00:30.339872,0 days 00:00:26.379242
7,2022-07-21 13:04:54.399324,2022-07-21 13:05:04.039013,0 days 00:00:09.639689
8,2022-07-21 14:44:35.157202,2022-07-21 14:47:03.444745,0 days 00:02:28.287543
9,2022-07-21 15:26:53.133385,2022-07-21 15:28:15.714353,0 days 00:01:22.580968


# General notes on the July measurement campain
## Barometers and Barometer-Themometers

### July 21 - July 23
* Unrealistic pressure flactuations of 30kPa are present.
* The signal on some sensors presents abnormal discontinuities: mean pressure can jump by 10kPa
in a discontinious manner.

Ex: 

index | start_time | end_time | duration | note
-- | --| -- | -- | --
8 | 2022-07-21 14:44:35.157202 |	2022-07-21 14:47:03.444745 |	0 days 00:02:28.287543 | Discontinutiy for sensor 10 and 13 at 14:45:45, after which the magnitude of flactuions degreseases significantly. 
27 |	2022-07-22 23:01:07.369285 |	2022-07-22 23:06:07.340935 |	0 days 00:04:59.971650 | Operating, no AoA reversal, no unrealistic fluctions, discontinuities on several sensors
40 |	2022-07-23 12:01:08.537584 |	2022-07-23 12:06:08.628855 |	0 days 00:05:00.091271 | Fluctions are modulated from extremee to reasonable.

### July 30
index | start_time | end_time | duration | note
-- | --| -- | -- | --
44 |	2022-07-30 10:07:42.098652 |	2022-07-30 10:12:22.180432 |	0 days 00:04:40.081780 | Extreme Pressure fluction modulation
46 |	2022-07-30 12:07:42.233502 |	2022-07-30 12:12:22.273700 |	0 days 00:04:40.040198 | Pressure side and suction side switch during rotation

## Turbine operation:

index | start_time | end_time | duration | note
-- | --| -- | -- | --
1|	2022-07-21 09:18:33.627776|	2022-07-21 09:19:11.496264|	0 days 00:00:37.868488| Rotating
...
6|	2022-07-21 13:00:03.960630|	2022-07-21 13:00:30.339872|	0 days 00:00:26.379242| Rotating
7|	2022-07-21 13:04:54.399324|	2022-07-21 13:05:04.039013|	0 days 00:00:09.639689| Stopped
8|	2022-07-21 14:44:35.157202|	2022-07-21 14:47:03.444745|	0 days 00:02:28.287543|Rotating
...
14| 2022-07-22 09:24:45.041385 |	2022-07-22 09:31:06.936444 |	0 days 00:06:21.895059 | Operating at almost 1 Hz (64 rpm is max for aventa)
16 | 	2022-07-22 12:17:36.417765  | 	2022-07-22 12:23:27.383299  | 	0 days 00:05:50.965534  | Stopping at 12:20
17  | 	2022-07-22 13:01:06.677570  | 	2022-07-22 13:06:06.477508  | 	0 days 00:04:59.799938  |  Starting
18 	 | 2022-07-22 14:01:06.561198  | 	2022-07-22 14:06:06.738049  | 	0 days 00:05:00.176851  | Rotating 0.5-0.6 Hz
19  | 	2022-07-22 15:01:06.628489  | 	2022-07-22 15:06:06.628454  | 	0 days 00:04:59.999965  |  Rotating with high accelerations and vibrations
20  | 	2022-07-22  |  16:01:06.701719  | 	2022-07-22 16:06:06.803389  | 	0 days 00:05:00.101670  | Rotating 0.5-0.6 Hz
...
23 |	2022-07-22 19:01:06.963738 |	2022-07-22 19:06:06.929668 |	0 days 00:04:59.965930 | Rotating 0.5-0.6 Hz
24 |	2022-07-22 20:01:07.099464 |	2022-07-22 20:06:07.167476 |	0 days 00:05:00.068012 | Stopped
...
26 |	2022-07-22 22:01:07.254009 |	2022-07-22 22:06:07.384462 |	0 days 00:05:00.130453 | Stoppped
27 |	2022-07-22 23:01:07.369285 |	2022-07-22 23:06:07.340935 |	0 days 00:04:59.971650 | Rotating
...
31 |	2022-07-23 03:01:07.748110 |	2022-07-23 03:06:07.829399 |	0 days 00:05:00.081289 | Rotating, but w sensor signal losses
...
35 |	2022-07-23 04:01:07.825897 |	2022-07-23 04:06:07.972167 |	0 days 00:05:00.146270 | Rotating, but w sensor signal losses
36 |	2022-07-23 05:01:07.913583 |	2022-07-23 05:06:08.178469 |	0 days 00:05:00.264886 | Stopped
37 |	2022-07-23 07:01:08.077070 |	2022-07-23 07:06:08.252841 |	0 days 00:05:00.175771 | Rotating 0.5-0.6 Hz
...
40 |	2022-07-23 12:01:08.537584 |	2022-07-23 12:06:08.628855 |	0 days 00:05:00.091271 | Rotating 0.5-0.6 Hz
41 |	2022-07-23 15:01:08.716257 |	2022-07-23 15:06:08.791163 |	0 days 00:05:00.074906 | Stopped
...
44 |	2022-07-30 10:07:42.098652 |	2022-07-30 10:12:22.180432 |	0 days 00:04:40.081780 | Rotating 0.5-0.6 Hz
...
49 |	2022-07-30 15:07:42.540008 | 	2022-07-30 15:12:22.550179 |	0 days 00:04:40.010171 | Rotating 0.5-0.6 Hz
50 |	2022-07-30 16:07:42.622773 |	2022-07-30 16:12:22.660772 |	0 days 00:04:40.037999 | Starting 
51 |	2022-07-30 17:07:42.733378 |	2022-07-30 17:12:22.770113 |	0 days 00:04:40.036735 | Rotating
52  | 	2022-07-30 18:07:42.801240  | 	2022-07-30 18:12:22.852727  | 	0 days 00:04:40.051487 | Stopped but some blade movement (some wind present)
...
56 |	2022-07-30 22:07:43.160687 	| 2022-07-30 22:12:23.115859 |	0 days 00:04:39.955172 | Stopped, used to get P_atm, Not sure if there is wind









# Data Playground
Lets play with Raw data

In [ ]:
df, data_limit_applied = client.get_sensor_data(
    installation_reference="aventa-turbine-test",
    node_id="2",
    sensor_type_reference="barometer",
    start=dt.datetime(2022, 7, 21, 10, 39, 00),
    finish=dt.datetime(2022, 7, 21, 10, 42, 00),
    row_limit=10000,
)

In [ ]:
data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
signal_df = df[["datetime"]+data_columns].set_index('datetime')
barometer = RawSignal(signal_df, "barometer")

In [ ]:
barometer.pad_gaps(dt.timedelta(seconds=1))

In [ ]:
barometer.measurement_to_variable()


There seems to be loads of non-sensical values.  Lets remove everything more than 1.5 Atm and less than 0.6  (... that's my arbitrary choice now, before I can come up with something better)

In [ ]:
barometer.dataframe = barometer.dataframe[(barometer.dataframe <= 150e3) & (barometer.dataframe >= 60e3)]

Lets plot just around peak pressure, so that we can look at the data..

In [ ]:
plot_start = dt.datetime(2022, 7, 21, 10, 41, 51, 150000)
plot_finish = dt.datetime(2022, 7, 21, 10, 41, 51, 650000)
time_mask = ((barometer.dataframe.index > plot_start) & (barometer.dataframe.index < plot_finish))
pre_processed_df = barometer.dataframe[time_mask].reset_index()

# Application and Plotting
Lets apply what we learned and make some nice plots.


In [7]:
sample_start = dt.datetime(2022, 7, 22, 23)
sample_finish = dt.datetime(2022, 7, 31)

In [46]:
df, data_limit_applied = client.get_sensor_data(
    installation_reference="aventa-turbine-test",
    node_id="2",
    sensor_type_reference="barometer_thermometer",
    start=sample_start,
    finish=sample_finish,
    row_limit=100,
)

data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
signal_df = df[["datetime"]+data_columns].set_index('datetime')
signal_df.columns = sensor_types_metadata["barometer_thermometer"]["sensors"]
thermometer = RawSignal(signal_df, "barometer_thermometer")
thermometer.pad_gaps(dt.timedelta(seconds=1))
thermometer.measurement_to_variable()


df, data_limit_applied = client.get_sensor_data(
    installation_reference="aventa-turbine-test",
    node_id="2",
    sensor_type_reference="barometer",
    start=sample_start,
    finish=sample_finish,
    row_limit=100,
)

data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
signal_df = df[["datetime"]+data_columns].set_index('datetime')
signal_df.columns = sensor_types_metadata["barometer"]["sensors"]
barometer = RawSignal(signal_df, "barometer")
barometer.pad_gaps(dt.timedelta(seconds=1))
barometer.measurement_to_variable()



We can filter out unreasonable temperature readings:

In [61]:
barometer.dataframe = barometer.dataframe[(barometer.dataframe <= 150e3) & (barometer.dataframe >= 60e3)]

In [47]:
thermo_filter = ((thermometer.dataframe <= 45) & (thermometer.dataframe >= 5))
thermometer.dataframe = thermometer.dataframe[thermo_filter]

We can filter baro data with thermo filter as it is more obvious from temperature when the data is reasonable.

In [60]:
thermo_filter.columns=barometer.dataframe.columns 
barometer_processed = RawSignal(barometer.dataframe[thermo_filter], "barometer")

In [62]:
barometer.plot(sensor_types_metadata)

In [51]:
thermometer.plot(sensor_types_metadata)

Get atm. pressure in still conditions

In [40]:
still_sample_start = dt.datetime(2022, 7, 30, 22, 8, 30)
still_sample_finish = dt.datetime(2022, 7, 31)

df, data_limit_applied = client.get_sensor_data(
    installation_reference="aventa-turbine-test",
    node_id="2",
    sensor_type_reference="barometer",
    start=still_sample_start,
    finish=still_sample_finish,
    row_limit=3000,
)

data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
signal_df = df[["datetime"]+data_columns].set_index('datetime')
signal_df.columns = sensor_types_metadata["barometer"]["sensors"]
still_barometer = RawSignal(signal_df, "barometer")
still_barometer.pad_gaps(dt.timedelta(seconds=1))
still_barometer.measurement_to_variable()

In [ ]:
resampled_df = still_barometer.dataframe.resample('1S').mean()

In [ ]:
plots.plot_sensors(still_barometer.dataframe.reset_index())

In [ ]:
plots.plot_sensors(resampled_df.reset_index())

In [64]:
atm_pressure=still_barometer.dataframe.mean()
barometer_processed.dataframe -= atm_pressure

In [ ]:
barometer_processed.dataframe -= 96.2e3

In [28]:
df, data_limit_applied = client.get_sensor_data(
    installation_reference="aventa-turbine-test",
    node_id="2",
    sensor_type_reference="accelerometer",
    start=sample_start,
    finish=sample_finish,
    row_limit=100,
)

data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
signal_df = df[["datetime"]+data_columns].set_index('datetime')
signal_df.columns = sensor_types_metadata["accelerometer"]["sensors"]
accelerometer = RawSignal(signal_df, "accelerometer")
accelerometer.pad_gaps(dt.timedelta(seconds=1))
accelerometer.measurement_to_variable()

In [29]:
accelerometer.plot(sensor_types_metadata)

In [ ]:
barometer_coordinates = pd.DataFrame(
{
    "x":[0.0922119084253499,0.0784111428350326,0.0635467289600351,0.0494718121475968,0.0348007242384172,0.0209836333929474,0.00821066059356812,0.000287016245904044,0.00404525690786845,0.0144832434319023,0.0262233800596873,0.0400800665711656,0.0527383343496627,0.067091235212706,0.0806198869961322,0.0950907543239808,0.109385753674316,0.124095983654786,0.137965865257593,0.152444428050515,0.166519849421778,0.180888288065098,0.194997584196934,0.209244523125562,0.205889294568664,0.191642397539166,0.177546273531482,0.16341218630832,0.149101720253823,0.13520156795253,0.120998714181473, 0.106888086108249, 0.317522175046627,0.303212316451663,0.289521584134576,0.27539208898574,0.261557829260995,0.247306755467187,0.233510956423097,0.219404694503124],
    "y":[-0.0298805503559557,-0.0289602925497801,-0.0286682644007734,-0.0282494451123373,-0.0283085310605089,-0.0220526447236176,-0.0150202186794781,-0.00153116363587266,0.0122178780781479,0.0233181734120601,0.0297669723860122,0.0366470139798434,0.0406771795779223,0.0451941341237041,0.0467644784370817,0.0496846161412822,0.0498060778947494,0.0513140914378661,0.0504517682647457,0.050052270897714,0.049185007882442,0.0476147087689916,0.0462310698726144,0.0442390615811808,-0.0255947526614279,-0.0259384688248189,-0.0270997119255558,-0.0275706114026977,-0.0290218042880069,-0.0288830613850707,-0.0292499894120214, -0.0294782194547466, -0.0172704802584763,-0.017977126230662,-0.0191442611821929,-0.0200474513720925,-0.0210341817018639,-0.0218470990312465,-0.0230808054519725,-0.0238661186683629],
    "sensor": barometer_processed.dataframe.columns.str.removeprefix("Abs. Baro. Sensor. ")
 })

In [67]:
import plotly.express as px

fig = px.scatter(barometer_coordinates, x="x", y="y", text="sensor")
fig.show()

In [ ]:
x_pressure_side = barometer_coordinates[barometer_coordinates["y"]<0]["x"].sort_values()
x_suction_side = barometer_coordinates[barometer_coordinates["y"]>0]["x"].sort_values()

suction_side_barometers = barometer_processed.dataframe.columns[x_suction_side.index]
pressure_side_barometers = barometer_processed.dataframe.columns[x_pressure_side.index]

In [ ]:
pressure_side_barometers

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "colab"

ani_samples = 500

#ani_start=dt.datetime(2022, 7, 22, 23, 2, 30)
#aero_pressure=aero_pressure[aero_pressure.index>ani_start]

suction_df = barometer_processed.dataframe[suction_side_barometers]
pressure_df = barometer_processed.dataframe[pressure_side_barometers]

#pressure_df = pressure_df.drop(["f19_", "f29_"], axis=1)
#pressure_df = pressure_df.drop(["f2_"], axis=1)

plot_frames = [go.Frame(
    data=[
        go.Scatter(            
            x=x_pressure_side,
            y=pressure_df.iloc[i],
            text=pressure_df.columns,
            name="Pressure side ",
            mode='lines+markers+text',
            marker=dict(color="red")
        ),
        go.Scatter(            
            x=x_suction_side,
            y=suction_df.iloc[i],
            text=suction_df.columns,
            name="Suction side ",
            mode='lines+markers+text',
            marker=dict(color="blue")
        )        
    ],
    name=str(suction_df.index[i])
    )
for i in range(ani_samples)]


sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Time:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 60},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": [{"args": [
        [str(suction_df.index[i])],
        {"frame": {"duration": 60, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 0}}
    ],
        "label": str(suction_df.index[i]),
        "method": "animate"} for i in range(ani_samples)]
}

fig = go.Figure(
data=plot_frames[0]["data"],
layout=go.Layout(
        title_text="Cp Curve", hovermode="closest",
        xaxis=dict(title="Chordwise sensor postition [m/1m]", range=[0, 0.35], autorange=False),
        yaxis=dict(title="Aerodynamic Pressure [Pa]", range=[2.5e3, -1e3], autorange=False),
        sliders=[sliders_dict],
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="\u25B6",
                                        method="animate",
                                        args=[None,
                                              {"frame": {"duration": 60, "redraw": True},
                                               "transition": {"duration": 0}}]),
                                   dict(label="Pause",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}])
                                   ],
                          yanchor="bottom",
                          y=-0.5,
                          )]),
frames = plot_frames

)


fig.show()